1. Load a tick list
2. For each climb tick compare its grade to the previous hardest climb for that month
3. Generate a list of months and their corresponding hardest grades by doing above

ryan's tick page : 
https://www.mountainproject.com/user/200200801/ryan-westby/ticks

In [61]:
import sqlite3 as sl
import requests
from bs4 import BeautifulSoup
import json
import re
import numpy as np
from datetime import datetime

con = sl.connect('my-test.db')

In [95]:
import pandas as pd
with con:
    df = pd.read_sql_query('SELECT ticks.url, grade, date FROM ticks INNER JOIN routes ON routes.url = ticks.url where ticks.rowid < 10;', con)
    
df.loc[df['grade'] == 5.1, 'grade'] = '5.10b/c'
df.loc[df['grade'] == 5.11, 'grade'] = '5.11b/c'
df.loc[df['grade'] == 5.12, 'grade'] = '5.12b/c'

df

,url,grade,date
0,https://www.mountainproject.com/route/10574098...,5.8,"Mar 9, 2021"
1,https://www.mountainproject.com/route/10580357...,5.11a,"Mar 9, 2021"
2,https://www.mountainproject.com/route/10939993...,5.12c,"Dec 5, 2020"
3,https://www.mountainproject.com/route/11118300...,5.11c,"Nov 28, 2020"
4,https://www.mountainproject.com/route/11194545...,5.10b,"Oct 19, 2020"
5,https://www.mountainproject.com/route/10588924...,5.10a/b,"Oct 17, 2020"
6,https://www.mountainproject.com/route/10579178...,5.10a,"Oct 17, 2020"
7,https://www.mountainproject.com/route/10590181...,5.9,"Oct 16, 2020"
8,https://www.mountainproject.com/route/10956504...,5.10a/b,"Oct 16, 2020"


In [94]:
# input a grade and a previous PBest, compare
# return true if new grade is higher, false if not

#def compareGrades(grade, highgrade):
#    if (len(grade) == 3 and len(highgrade) == 3) or (len(grade) > 3 and len(highgrade) > 3):
#        return grade > highgrade
#    if len(grade) == 3 and len(highgrade) > 3:
#        return False
#    if len(grade) > 3 and len(highgrade) == 3:
#        return True


# add a new column to master df, monthyear that's a formatted version of date, gradekey

df.at[0,'monthyear'] = ''
df.at[0,'gradekey'] = 0.0
for index, row in df.iterrows(): # for every tick in the ticklist
    string = datetime.strptime(row.date, '%b %d, %Y')
    df.at[index,'monthyear'] = string.strftime('%Y-%m') # add monthyear value
    month = row.monthyear

    if str(row.grade) in gradeToNum.keys(): # also, for every tick, if the grade is in the translator
        df.at[index, 'gradekey'] = gradeToNum[str(row.grade)] # insert the new gradekey in the column
    else:
        df.at[index,'gradekey'] = 0 # if it's not in the translator it's probably an iceclimb or something, skip it
    
    
    
    
#the goal is to make a dictionary of monthly PBs
def monthlyPBs(df):
    monthlyPBs = {} # monhigh is a list of months and monthly PB
    for index, row in df.iterrows(): # for every climb in the df
        month = row.monthyear
        grade = row.gradekey
        
        if month not in monthlyPBs.keys(): # if it's the first climb of the month
            monthlyPBs[month] = grade # set the high of the month to the climb grade
        else: # if it's not the first
            highgrade = monthlyPBs[month] # get the current PB of the month to compare
            if grade > highgrade: # if the current PB is higher
                monthlyPBs[month] = grade # set a new current PB 

        if str(row.grade) in gradeToNum.keys():
            df.at[index, 'gradekey'] = gradeToNum[str(row.grade)]
        else:
            df.at[index,'gradekey'] = 0.0
    return monthlyPBs

monthlyPBs = monthlyPBs(df)
monthlyPBs

{'2021-03': 15.0, '2020-12': 21.0, '2020-11': 17.0, '2020-10': 11.0}

In [101]:
# required for getting datefill

highlist = list(monthlyPBs.items())

def Sort_Tuple(tup):  
    tup.sort(key = lambda x: x[0])  
    return tup  

highlist = Sort_Tuple(highlist)

datefill = pd.date_range(highlist[0][0],highlist[-1][0], freq='MS').strftime("%Y-%m").tolist()
print(datefill)

['2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03']


In [97]:
def recentHighs(gradeDict):
    tickedMonths = list(gradeDict.keys())
    list.sort(tickedMonths)
    
    lastGrade = ''
    recentGradeList = []
    
    # for month in datefill, check if the month is in the tickedMonths list 
    # if it is, get the grade 
    # append it to the list, upgrade lastGrade
    # if it isn't in tickedMonths, use lastGrade
    
    for month in datefill: 
        if month in tickedMonths:
            recentGradeList.append(gradeDict[month])
            lastGrade = gradeDict[month]
        else:
            recentGradeList.append(lastGrade)
    
    return recentGradeList
    
recentGradeList = recentHighs(monhigh)

print(recentGradeList)    

['2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03']
2020-10
2020-11
2020-12
2021-01
2021-02
2021-03
[10.0, 17.0, 21.0, 21.0, 21.0, 15.0]


In [100]:
# this function finds a list of incrementing grades for whatever was the highest at that point

datefill = pd.date_range(highlist[0][0],highlist[-1][0], freq='MS').strftime("%Y-%m").tolist()
print(datefill)
highGradeList = []

# check if there is a grade associated for every month 
# check if that grade is higher than the previous high grade
# if it is, append that grade and set new highest grade to that
# if not, append the previous high grade and continue

def highGrade(gradeDict):
    tickedMonths = list(gradeDict.keys())
    list.sort(tickedMonths)
    
    highGrade = 0.0
    for month in datefill: # for every month
        if month in tickedMonths: # check if there is a climbing grade in the ticked maxes
            grade = gradeDict[month]
            if grade > highGrade:
                highGrade = grade
                highGradeList.append(highGrade) # add it to the list
            else:
                highGradeList.append(highGrade)
        else:
            highGradeList.append(highGrade)

highGrade(monhigh)

highGradeList

['2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03']


[10.0, 17.0, 21.0, 21.0, 21.0, 21.0]

what's next? you have a list of months, and a list of grades 
get the plot up
add a second plot
change the months to the time module 


but first we need a function that translates grades from ordinal data to something js can understand like count scale data

when does it make sense to do the translating ?? in the python phase ? does it matter? it's kind of the same no matter where you do it. you'll have to translate back at some point. the crux is that you want to translate the least amount of times. this means doing it in the pandas df as soon as you get it, then back into human grades in the plot


1 create a dictionary with integers corresponding to all the grades 
2 iterate over all the grades in the df and translate them


In [52]:
numToGrade = {
    0: ('5.0', 'easy', 'Easy'),    1: ('5.1'),    2: ('5.2'),    3: ('5.3'),    4: ('5.4'),    5: ('5.5'),    6: ('5.6'),    7: ('5.7-'),    8: ('5.7'),    9: ('5.7+'),    10: ('5.8-'),    11: ('5.8'),    12: ('5.8+'),    13: ('5.9-'),    14: ('5.9'),    15: ('5.9+'),    16: ('5.10a', '5.10-', '5.10a/b'),    17: ('5.10b'),    18: ('5.10b/c', '5.10'),    19: ('5.10c'),    20: ('5.10d', '5.10+', '5.10c/d'),    21: ('5.11a', '5.11-', '5.11a/b'),    22: ('5.11b'),    23: ('5.11b/c', '5.11'),    24: ('5.11c'),    25: ('5.11d', '5.11+', '5.11c/d'),    21: ('5.12a', '5.12-', '5.12a/b'),    22: ('5.12b'),    23: ('5.12b/c', '5.12'),    24: ('5.12c'),    25: ('5.12d', '5.12+', '5.12c/d'),    26: ('5.13a', '5.13-', '5.13a/b'),    27: ('5.13b'),    28: ('5.13b/c', '5.13'),    29: ('5.13c'),    30: ('5.13d', '5.13+', '5.13c/d'),
    31: ('5.14a', '5.14-', '5.14a/b'),    32: ('5.14b'),    33: ('5.14b/c', '5.14'),    34: ('5.14c'),    35: ('5.14d', '5.14+', '5.14c/d'),    36: ('5.15a', '5.15-', '5.15a/b'),    37: ('5.15b'),    38: ('5.15b/c', '5.15'),    39: ('5.15c'),    40: ('5.15d', '5.15+', '5.15c/d'),    41: ('5.16a', '5.16-', '5.16a/b'),    42: ('5.16b'),    43: ('5.16b/c', '5.16'),   44: ('5.16c'),    45: ('5.16d', '5.16+', '5.16c/d'),
    47: ('5.16b'),    48: ('5.16b/c', '5.16'),    49: ('5.16c'),    50: ('5.16d', '5.16+'),        51: ('VB', 'V-easy'),    52: ('V0-'),    53: ('V0'),    54: ('V0+', 'V0-1'),    55: ('V1-'),    56: ('V1'),    57: ('V1+', 'V1-2'),    58: ('V2-'),
    59: ('V2'),    60: ('V2+', 'V2-3'),    61: ('V3-'),    62: ('V3'),    63: ('V3+', 'V3-4'),    64: ('V4-'),    65: ('V4'),    66: ('V4+', 'V4-5'),    67: ('V5-'),    68: ('V5'),    69: ('V5+', 'V5-6'),    70: ('V6-'),    71: ('V6'),    72: ('V6+', 'V6-7'),
    73: ('V7-'),    74: ('V7'),    75: ('V7+', 'V7-8'),    76: ('V8-'),    77: ('V8'),    78: ('V8+', 'V8-9'),    79: ('V9-'),    80: ('V9'),    81: ('V9+', 'V9-10'),    82: ('V10-'),    83: ('V10'),    84: ('V10+', 'V10-11'),
    85: ('V11-'),    86: ('V11'),    87: ('V11+', 'V11-12'),    88: ('V12-'),    89: ('V12'),    90: ('V12+', 'V12-13'),    91: ('V13-'),    92: ('V13'),    93: ('V13+', 'V13-14'),    94: ('V14-'),    95: ('V14'),    96: ('V14+', 'V14-15'),
    97: ('V15-'),    98: ('V15'),    99: ('V15+', 'V15-16'),    100: ('V16-'),    101: ('V16'),    102: ('V16+', 'V16-17'),    103: ('V17-'),    104: ('V17'),    105: ('V17+', 'V17-18')
}

gradeToNum = {y:x for x,y in numToGrade.items()}

gradeToNum


     

{('5.0', 'easy', 'Easy'): 0,
 '5.1': 1,
 '5.2': 2,
 '5.3': 3,
 '5.4': 4,
 '5.5': 5,
 '5.6': 6,
 '5.7-': 7,
 '5.7': 8,
 '5.7+': 9,
 '5.8-': 10,
 '5.8': 11,
 '5.8+': 12,
 '5.9-': 13,
 '5.9': 14,
 '5.9+': 15,
 ('5.10a', '5.10-', '5.10a/b'): 16,
 '5.10b': 17,
 ('5.10b/c', '5.10'): 18,
 '5.10c': 19,
 ('5.10d', '5.10+', '5.10c/d'): 20,
 ('5.12a', '5.12-', '5.12a/b'): 21,
 '5.12b': 22,
 ('5.12b/c', '5.12'): 23,
 '5.12c': 24,
 ('5.12d', '5.12+', '5.12c/d'): 25,
 ('5.13a', '5.13-', '5.13a/b'): 26,
 '5.13b': 27,
 ('5.13b/c', '5.13'): 28,
 '5.13c': 29,
 ('5.13d', '5.13+', '5.13c/d'): 30,
 ('5.14a', '5.14-', '5.14a/b'): 31,
 '5.14b': 32,
 ('5.14b/c', '5.14'): 33,
 '5.14c': 34,
 ('5.14d', '5.14+', '5.14c/d'): 35,
 ('5.15a', '5.15-', '5.15a/b'): 36,
 '5.15b': 37,
 ('5.15b/c', '5.15'): 38,
 '5.15c': 39,
 ('5.15d', '5.15+', '5.15c/d'): 40,
 ('5.16a', '5.16-', '5.16a/b'): 41,
 '5.16b': 47,
 ('5.16b/c', '5.16'): 48,
 '5.16c': 49,
 ('5.16d', '5.16+', '5.16c/d'): 45,
 ('5.16d', '5.16+'): 50,
 ('VB', 'V-ea

In [59]:
gradeToNum = {
    '5.0' : 0, 'easy' : 0, 'Easy' : 0, '5.1' : 1, '5.2' : 2, '5.3' :  3, '5.4' : 4,
    '5.5' : 5, '5.6' : 6, '5.7-' : 7, '5.7' : 7, '5.7+' : 7, '5.8-' : 8, '5.8' : 8, 
    '5.8+' : 8, '5.9-' : 9, '5.9' : 9, '5.9+' : 9,'5.10a' : 10, '5.10-' : 10, 
    '5.10a/b' : 10, '5.10b' : 11, '5.10b/c' : 12, '5.10' : 12, '5.10c' : 13, 
    '5.10d' : 14, '5.10+' : 14, '5.10c/d' : 14, '5.11a' : 15, '5.11-' : 15, '5.11a/b' : 15,
    '5.11b' : 16,'5.11b/c' : 17, '5.11' : 17, '5.11c' : 18, '5.11d' : 19, '5.11+' : 19, 
    '5.11c/d' : 19, '5.12a' : 20, '5.12-' : 20, '5.12a/b' : 20, '5.12b' : 21, '5.12b/c' : 22, 
    '5.12' : 22, '5.12c' : 23, '5.12d' : 24, '5.12+' : 24, '5.12c/d' : 24, '5.13a' : 25, 
    '5.13-' : 25, '5.13a/b': 25, '5.13b': 26, '5.13b/c': 27, '5.13': 27, '5.13c': 28, 
    '5.13d': 29, '5.13+': 29, '5.13c/d': 29, '5.14a': 30, '5.14-': 30, '5.14a/b': 30, '5.14b': 31,
    '5.14b/c': 32, '5.14': 32, '5.14c': 33, '5.14d': 34, '5.14+': 34, '5.14c/d': 34,
    '5.15a': 35, '5.15-': 35, '5.15a/b': 35, '5.15b': 36, '5.15b/c': 37, '5.15': 37, '5.15c': 38,
    '5.15d': 39, '5.15+': 39, '5.15c/d': 39, '5.16a': 40, '5.16-': 40, '5.16a/b': 40, '5.16b': 41,
    '5.16b/c': 41, '5.16': 41, '5.16c': 42
}

boulderToNum = {
    'VB': 100, 'V-easy': 100, 'V0-': 101, 'V0':102, 'V0+': 103, 'V0-1': 103, 'V1-': 104, 'V1': 105, 
    'V1+': 106, 'V1-2':106, 'V2-':107, 'V2': 108, 'V2+': 109, 'V2-3': 110, 'V3-': 111, 'V3': 112,
    'V3+': 113, 'V3-4': 113, 'V4-': 114, 'V4':115, 'V4+': 116, 'V4-5':116, 'V5-':117,'V5':118,
    'V5+': 119, 'V5-6':119, 'V6-':120, 'V6': 121, 'V6+':122, 'V6-7':122,'V7-':123,'V7':124,
    'V7+':125, 'V7-8':125, 'V8-':126, 'V8':127, 'V8+':128, 'V8-9':128, 'V9-':129, 'V9': 130,
    'V9+':131, 'V9-10':131, 'V10-':132,'V10':133, 'V10+':134, 'V10-11':134,'V11-':135,'V11':136,
    'V11+':137, 'V11-12':137, 'V12-':138,'V12':139, 'V12+': 140, 'V12-13': 140, 'V13-':141, 
    'V13':142, 'V13+':143, 'V13-14':143, 'V14-':144, 'V14':145, 'V14+':146, 'V14-15':146,
    'V15-':147, 'V15':148, 'V15+':149, 'V15-16':149, 'V16-':150,'V16':151, 'V16+':152, 'V16-17':152,
    'V17-':153, 'V17':154, 'V17+': 155, 'V17-18':156
}

In [36]:
import pandas as pd
with con:
    df = pd.read_sql_query('SELECT grade, type FROM routes', con)
    
df.shape

(240019, 2)

In [56]:
gradeToNum.keys()

dict_keys([('5.0', 'easy', 'Easy'), '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7-', '5.7', '5.7+', '5.8-', '5.8', '5.8+', '5.9-', '5.9', '5.9+', ('5.10a', '5.10-', '5.10a/b'), '5.10b', ('5.10b/c', '5.10'), '5.10c', ('5.10d', '5.10+', '5.10c/d'), ('5.12a', '5.12-', '5.12a/b'), '5.12b', ('5.12b/c', '5.12'), '5.12c', ('5.12d', '5.12+', '5.12c/d'), ('5.13a', '5.13-', '5.13a/b'), '5.13b', ('5.13b/c', '5.13'), '5.13c', ('5.13d', '5.13+', '5.13c/d'), ('5.14a', '5.14-', '5.14a/b'), '5.14b', ('5.14b/c', '5.14'), '5.14c', ('5.14d', '5.14+', '5.14c/d'), ('5.15a', '5.15-', '5.15a/b'), '5.15b', ('5.15b/c', '5.15'), '5.15c', ('5.15d', '5.15+', '5.15c/d'), ('5.16a', '5.16-', '5.16a/b'), '5.16b', ('5.16b/c', '5.16'), '5.16c', ('5.16d', '5.16+', '5.16c/d'), ('5.16d', '5.16+'), ('VB', 'V-easy'), 'V0-', 'V0', ('V0+', 'V0-1'), 'V1-', 'V1', ('V1+', 'V1-2'), 'V2-', 'V2', ('V2+', 'V2-3'), 'V3-', 'V3', ('V3+', 'V3-4'), 'V4-', 'V4', ('V4+', 'V4-5'), 'V5-', 'V5', ('V5+', 'V5-6'), 'V6-', 'V6', ('V6+', 'V6-7'),

In [35]:
asdf=pd.unique(df.grade)

asdf

array([5.8, 5.11, '5.10c', 5.9, '5.10b/c', '5.10+', '5.11a', '5.10b',
       '5.10d', 5.7, '5.10c/d', '5.12b', '5.10-', 'V3', '5.12a', 'V2',
       '5.11d', '5.11c', '5.10a', 5.6, 5.5, '5.11b', 'WI3-4', 'C1', 5.1,
       '5.12d', '5.11a/b', '5.9+', '5.11b/c', '5.9-', 5.4, 'WI5', 'V6-7',
       'V11-12', '5.11-', '5.7+', '5.8-', '5.12c', '5.8+', '5.13a', 'V8',
       '5.11+', 'V4', 'Mod.', 'V0', 'V2-3', 'WI2+', 'V-easy', 'V6', 'V7',
       'V5', '5.11c/d', 'C2+', 'M10', 'V4+', 'WI4-', 'V5-', 'V0-1', 'M7',
       'WI2', 'V0-', 'V1-2', '5.12b/c', 'M4', 5.12, 'V5-6', 'V1', 'V6+',
       5.0, 'V2+', 'V9', 5.2, '5.10a/b', 'V0+', 'V8-', 'V11', 'V3-',
       '5.13c', '5.13b', 'V8-9', 'V1+', 'V1-', 'V7-8', 'A2+', 'M1',
       'V4-5', 'V3-4', 'M6+', '5.12-', '5.14a', 'M5', '5.12a/b', 'V2-',
       'V4-', 'WI3', '5.12c/d', 'V6-', 'M6-7', '3rd', 'A2-3', 5.3, '4th',
       'WI7', 'V5+', '5.13d', '5.14c', 'WI4-5', 'V3+', 'A5', 'WI4',
       '5.13+', 5.13, 'Easy', 'WI5+', 'V10', 'WI3+', 'M3+', 'WI4+'